<a href="https://colab.research.google.com/github/Zburns31/Code/blob/master/Predicting_Basketball_Scores_with_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary TODO:

Target: 
1.   Predict Winners of NCAA games
2.   Predict spreads of games



In [1]:
!pip install fuzzymatcher

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144798 sha256=231bcd65276b86e12b44c48e683b8aef7b484013f5bb328d798276a400131dda
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
  Created wheel for metaphone: filename=Metaphone-0.6-cp36-none-any.whl size=13906 sha256=556fe9d6ef2460f0e2b5d5d2deb59abc8d59b24c522d6236ca4cee44d964043d
  Stored in directory: /root/.cache/pip/wheels/4e/7c/f7/162d726fc83491ef23c7a0b989005024eb83a1408c96f32eaf
Successfully built python-Levenshtein metaphone


In [2]:
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
from fuzzymatcher import link_table, fuzzy_left_join
import re

In [3]:
teams = pd.read_csv('Teams.csv', header=None, names=['Conference', 'Team'])
teams

,Conference,Team
0,America East,Vermont
1,America East,Stony Brook
2,America East,UMBC
3,America East,Hartford
4,America East,Albany
...,...,...
348,Western Athletic,California Baptist
349,Western Athletic,Cal State Bakersfield
350,Western Athletic,Seattle
351,Western Athletic,UMKC


In [4]:
teams['Conference'].unique()

array(['America East', 'American Athletic Conference', 'Atlantic 10',
       'Atlantic Coast', 'Atlantic Sun', 'Big 12', 'Big East', 'Big Sky',
       'Big South', 'Big Ten', 'Big West', 'Colonial Athletic',
       'Conference USA', 'Horizon', 'Ivy League',
       'Metro Atlantic Athletic', 'Mid-American', 'Mid-Eastern',
       'Missouri Valley', 'Mountain West', 'Northeast', 'Ohio Valley',
       'Pac-12', 'Patriot League', 'Southeastern', 'Southern',
       'Southland', 'Southwestern Athletic', 'Summit League', 'Sun Belt',
       'West Coast', 'Western Athletic'], dtype=object)

In [5]:
# print(teams['Team'].unique())
print(len(teams['Team'].unique()))

353


In [6]:
games = pd.read_csv('Games.csv', header=None, 
                    names=['Home_Team', 'Home_Score', 'Away_Team', 'Away_Score'])
games.drop_duplicates(inplace=True)
games

,Home_Team,Home_Score,Away_Team,Away_Score
2015-11-13,Hawaii,87,Montana St.,76
2015-11-13,Eastern Mich.,70,Vermont,50
2015-11-13,Columbia,107,Kean,62
2015-11-13,La.-Monroe,88,McMurry,43
2015-11-13,Yale,70,Fairfield,57
...,...,...,...,...
2019-03-21,Hampton,81,St. Francis (B&#039;klyn),72
2019-03-21,La.-Monroe,87,Kent State,77
2019-03-21,CSU Fullerton,58,CSU Bakersfield,66
2019-03-22,Southern Utah,80,Drake,73


In [7]:
print(games.index.max())
print(games.index.min())

2019-03-22
2015-11-13


In [8]:
# Remove games with 0 score
games = games.loc[(games['Home_Score']>0) & (games['Away_Score']>0), :]

In [9]:
# print(games['Home_Team'].unique())
print(len(games['Home_Team'].unique()))

452


In [10]:
# games['Home_Team_clean'] = games['Home_Team'].map(lambda x: ''.join([i for i in x if i.isalpha()]))
games['Home_Team'] = games['Home_Team'].str.replace('[^A-Za-z]+',' ')
games['Away_Team'] = games['Away_Team'].str.replace('[^A-Za-z]+',' ')

games[(games['Home_Team'].str.isalnum()) | (games['Away_Team'].str.isalnum())]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Home_Team,Home_Score,Away_Team,Away_Score
2015-11-13,Hawaii,87,Montana St,76
2015-11-13,Eastern Mich,70,Vermont,50
2015-11-13,Columbia,107,Kean,62
2015-11-13,La Monroe,88,McMurry,43
2015-11-13,Yale,70,Fairfield,57
...,...,...,...,...
2019-03-20,Seattle U,68,Presbyterian,73
2019-03-20,Cal Baptist,55,LMU,56
2019-03-21,Hampton,81,St Francis B klyn,72
2019-03-22,Southern Utah,80,Drake,73


In [11]:
# print(games['Away_Team'].unique())
print(len(games['Away_Team'].unique()))

1021


In [12]:
left_on_home = ['Home_Team']
left_on_away = ['Away_Team']
right_on = ['Team']

games = games.reset_index().rename(columns={'index': 'Date'}) # Drop the index so we can perserve the date column

df = fuzzy_left_join(games, teams, left_on_home, right_on)
df.rename(columns={'Conference': 'Home_Team_Conference',
                   'best_match_score': 'home_team_match_score',
                   'Team': 'home_team_rec_match'}, inplace=True)

df.drop(['__id_left', '__id_right'], axis=1, inplace=True)

merged_df = fuzzy_left_join(df, teams, left_on_away, right_on)
merged_df.rename(columns={'Conference': 'Away_Team_Conference',
                          'best_match_score': 'away_team_match_score',
                          'Team': 'away_team_rec_match'}, 
                 inplace=True)

merged_df.drop(['__id_left', '__id_right'], axis=1, inplace=True)

# https://stackoverflow.com/questions/13636848/is-it-possible-to-do-fuzzy-match-merge-with-python-pandas
# df = link_table(games, teams, left_on, right_on)

In [13]:
merged_df.head(10)

,away_team_match_score,home_team_match_score,Date,Home_Team,Home_Score,Away_Team,Away_Score,Home_Team_Conference,home_team_rec_match,Away_Team_Conference,away_team_rec_match
0,0.095062,0.187595,2015-11-13,Hawaii,87,Montana St,76,Big West,Hawaii,Big Sky,Montana
8,0.187595,-0.006377,2015-11-13,Eastern Mich,70,Vermont,50,Ohio Valley,Eastern Illinois,America East,Vermont
9,NaN,0.187595,2015-11-13,Columbia,107,Kean,62,Ivy League,Columbia,NaN,NaN
10,NaN,0.022599,2015-11-13,La Monroe,88,McMurry,43,Sun Belt,Louisiana-Monroe,NaN,NaN
11,0.187595,0.171691,2015-11-13,Yale,70,Fairfield,57,Ivy League,Yale,Metro Atlantic Athletic,Fairfield
12,-0.044887,0.330115,2015-11-13,North Dakota,99,Minn Morris,69,Summit League,North Dakota,Northeast,Robert Morris
13,0.185960,0.003722,2015-11-13,Central Mich,89,Jacksonville St,83,Mid-American,Central Michigan,Atlantic Sun,Jacksonville
21,NaN,0.187595,2015-11-13,Army,93,Ferrum,54,Patriot League,Army,NaN,NaN
22,0.155787,0.071763,2015-11-13,Iowa St,68,Colorado,62,Big Ten,Iowa,Pac-12,Colorado
25,0.174221,0.339217,2015-11-13,Ole Miss,90,Northwestern St,76,Southeastern,Ole Miss,Big Ten,Northwestern


In [29]:
""" Logic:
- Remove fuzzy matches with low scores as most scores < 0 do not match correctly
- Remove teams where the conference is NULL, meaning they do not show up in the teams file
- Make sure that the matching record for the home team and the away team is not the same

Rutgers gets matched with Rutgers -newark 
                     so we need to eliminate cases where the same match is found 
"""
clean_df = None
clean_df = merged_df.loc[(merged_df['home_team_match_score'] > 0) & 
                         (merged_df['away_team_match_score'] > 0) & 
                         (merged_df['Home_Team_Conference'].notnull()) & 
                         (merged_df['Away_Team_Conference'].notnull()) &
                         (merged_df['home_team_rec_match'] != merged_df['away_team_rec_match']), :]

clean_df.head(10)

,away_team_match_score,home_team_match_score,Date,Home_Team,Home_Score,Away_Team,Away_Score,Home_Team_Conference,home_team_rec_match,Away_Team_Conference,away_team_rec_match
0,0.095062,0.187595,2015-11-13,Hawaii,87,Montana St,76,Big West,Hawaii,Big Sky,Montana
11,0.187595,0.171691,2015-11-13,Yale,70,Fairfield,57,Ivy League,Yale,Metro Atlantic Athletic,Fairfield
13,0.185960,0.003722,2015-11-13,Central Mich,89,Jacksonville St,83,Mid-American,Central Michigan,Atlantic Sun,Jacksonville
22,0.155787,0.071763,2015-11-13,Iowa St,68,Colorado,62,Big Ten,Iowa,Pac-12,Colorado
25,0.174221,0.339217,2015-11-13,Ole Miss,90,Northwestern St,76,Southeastern,Ole Miss,Big Ten,Northwestern
33,0.342265,0.079303,2015-11-13,Wright St,77,South Dakota,69,Horizon,Wright State,Summit League,South Dakota
35,0.365155,0.145897,2015-11-13,Penn,76,Robert Morris,75,Big Ten,Penn State,Northeast,Robert Morris
36,0.187595,0.365155,2015-11-13,Sacred Heart,76,Quinnipiac,64,Northeast,Sacred Heart,Metro Atlantic Athletic,Quinnipiac
37,0.282657,0.447723,2015-11-13,St John s NY,66,Wagner,57,Big East,St John's,Northeast,Wagner
50,0.164296,0.155787,2015-11-13,Duke,92,Siena,74,Atlantic Coast,Duke,Metro Atlantic Athletic,Siena


In [30]:
# Check if there are any ties
clean_df[clean_df['Home_Score'] == clean_df['Away_Score']]

,away_team_match_score,home_team_match_score,Date,Home_Team,Home_Score,Away_Team,Away_Score,Home_Team_Conference,home_team_rec_match,Away_Team_Conference,away_team_rec_match


In [31]:
# Check if there are any games with no score
games[(games['Home_Score'] == 0) & (games['Away_Score']==0)]

,Date,Home_Team,Home_Score,Away_Team,Away_Score


In [32]:
games[(games['Home_Score'] == 0) | (games['Away_Score']==0)]

,Date,Home_Team,Home_Score,Away_Team,Away_Score


In [33]:
clean_df['point_spread'] = clean_df['Home_Score'] - clean_df['Away_Score']
clean_df['winner'] = np.where(clean_df['Home_Score'] > clean_df['Away_Score'], 
                              clean_df['Home_Team'], 
                              clean_df['Away_Team']
)

clean_df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,away_team_match_score,home_team_match_score,Date,Home_Team,Home_Score,Away_Team,Away_Score,Home_Team_Conference,home_team_rec_match,Away_Team_Conference,away_team_rec_match,point_spread,winner
0,0.095062,0.187595,2015-11-13,Hawaii,87,Montana St,76,Big West,Hawaii,Big Sky,Montana,11,Hawaii
11,0.187595,0.171691,2015-11-13,Yale,70,Fairfield,57,Ivy League,Yale,Metro Atlantic Athletic,Fairfield,13,Yale
13,0.185960,0.003722,2015-11-13,Central Mich,89,Jacksonville St,83,Mid-American,Central Michigan,Atlantic Sun,Jacksonville,6,Central Mich
22,0.155787,0.071763,2015-11-13,Iowa St,68,Colorado,62,Big Ten,Iowa,Pac-12,Colorado,6,Iowa St
25,0.174221,0.339217,2015-11-13,Ole Miss,90,Northwestern St,76,Southeastern,Ole Miss,Big Ten,Northwestern,14,Ole Miss
33,0.342265,0.079303,2015-11-13,Wright St,77,South Dakota,69,Horizon,Wright State,Summit League,South Dakota,8,Wright St
35,0.365155,0.145897,2015-11-13,Penn,76,Robert Morris,75,Big Ten,Penn State,Northeast,Robert Morris,1,Penn
36,0.187595,0.365155,2015-11-13,Sacred Heart,76,Quinnipiac,64,Northeast,Sacred Heart,Metro Atlantic Athletic,Quinnipiac,12,Sacred Heart
37,0.282657,0.447723,2015-11-13,St John s NY,66,Wagner,57,Big East,St John's,Northeast,Wagner,9,St John s NY
50,0.164296,0.155787,2015-11-13,Duke,92,Siena,74,Atlantic Coast,Duke,Metro Atlantic Athletic,Siena,18,Duke


In [36]:
def avg_points_per_last_n_games(df, n_games = [5,10,20,30]):
  """ Function to calculate the average number of points a team scores over the last N number of games
  """
  for i in n_games:
    df[f'last_{str(i)}_home_games_avg_points'] = df['Home_Score'].rolling(window = i, min_periods = 1).mean()
    # df[f'last_{str(i)}_away_games_avg_points'] = df['Away_Score'].rolling(window = i, min_periods = 1).mean()

  return df


clean_df['home_row_number'] = clean_df.groupby(by=['Home_Team']).cumcount()
# clean_df['away_row_number'] = clean_df.groupby(by=['Away_Team']).cumcount()
yale_df = clean_df.loc[(clean_df['Home_Team']=='Yale')]

res = avg_points_per_last_n_games(yale_df)
res.head(20)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,away_team_match_score,home_team_match_score,Date,Home_Team,Home_Score,Away_Team,Away_Score,Home_Team_Conference,home_team_rec_match,Away_Team_Conference,away_team_rec_match,point_spread,winner,home_row_number,last_5_home_games_avg_points,last_10_home_games_avg_points,last_20_home_games_avg_points,last_30_home_games_avg_points
11,0.187595,0.171691,2015-11-13,Yale,70,Fairfield,57,Ivy League,Yale,Metro Atlantic Athletic,Fairfield,13,Yale,0,70.000000,70.000000,70.000000,70.000000
730,0.365155,0.171691,2015-11-16,Yale,99,Sacred Heart,77,Ivy League,Yale,Northeast,Sacred Heart,22,Yale,1,84.500000,84.500000,84.500000,84.500000
3432,0.115131,0.171691,2015-12-02,Yale,79,Bryant,40,Ivy League,Yale,Northeast,Bryant University,39,Yale,2,82.666667,82.666667,82.666667,82.666667
3822,0.187595,0.171691,2015-12-05,Yale,72,Vermont,54,Ivy League,Yale,America East,Vermont,18,Yale,3,80.000000,80.000000,80.000000,80.000000
7661,0.187595,0.171691,2016-01-06,Yale,83,NJIT,65,Ivy League,Yale,Atlantic Sun,NJIT,18,Yale,4,80.600000,80.600000,80.600000,80.600000
9018,0.187595,0.171691,2016-01-16,Yale,77,Brown,68,Ivy League,Yale,Ivy League,Brown,9,Yale,5,82.000000,80.000000,80.000000,80.000000
10736,0.145897,0.171691,2016-01-29,Yale,81,Penn,58,Ivy League,Yale,Big Ten,Penn State,23,Yale,6,78.400000,80.142857,80.142857,80.142857
11065,0.187595,0.171691,2016-01-30,Yale,79,Princeton,75,Ivy League,Yale,Ivy League,Princeton,4,Yale,7,78.400000,80.000000,80.000000,80.000000
11700,0.187595,0.171691,2016-02-05,Yale,86,Columbia,72,Ivy League,Yale,Ivy League,Columbia,14,Yale,8,81.200000,80.666667,80.666667,80.666667
12026,0.187595,0.171691,2016-02-06,Yale,83,Cornell,52,Ivy League,Yale,Ivy League,Cornell,31,Yale,9,81.200000,80.900000,80.900000,80.900000


In [ ]:
yale_df[yale_df('Away')]